# Курсовой проект

По дисциплине "Интеллектуальный анализ и обработка данных".

Выполнили: ст. гр. 4230М А.Н. Секунов, Д.А. Фомин.


## Задание

1.
    1. Для вариантов 1-20 дан многомерный набор данных телеметрической информации (ТМИ) малого космического аппарата, полученный в результате  предобработки и разметки в ЛР3. Разработать приложение на языке Python и  необходимых библиотек машинного и глубокого обучения на основе  классификационной модели, заданной в варианте, которое определяет техническое  состояние малого космического аппарата: штатное ф-е, отказ, сбой, на основе  данных его ТМИ.
    2. Для вариантов 21-40 дан многомерный набор данных сетевого трафика. Разработать приложение на языке Python и необходимых библиотек машинного и глубокого обучения на основе классификационной модели, заданной в варианте, х, которое определяет попытки сетевых вторжений на основе данных сетевого трафика.
2. Выполнить разведочный анализ данных как в ЛР.
3. Анализ выполняется для исходного набора данных и преобразованного исходного набора данных, который Вы подбираете самостоятельно с целью получения лучшей точности классификации.
4. Проверить набор данных на сбалансированность классов. В случае несбалансированного набора данных помимо “Classification report” с основными метриками: Accuracy, Precision, Recall, F1-score, также оценивать balanced_accuracy_score. Также оценивать и анализировать ROC_AUC_score.
5. Для выбранной комбинации набора данных и предиктивной модели выполняется поиск гиперпараметров с помощью генетического алгоритма. Реализовывать ГА можно самостоятельно или воспользоваться библиотекой DEAP (https://deap.readthedocs.io/en/master/index.html). 
  Для выбранной комбинации набора данных и предиктивной модели выполняется поиск гиперпараметров методом greedsearch. Сравнить полученные результаты.
  Во все НС классификационные модели добавить слой регуляризации Dropuot().
  Сохранять лог всех генерируемых НС моделей в популяции.
  Гиперпараметры: число сверточных фильтров в сверточных слоях, размер свертки в сверточных слоях, число нейронов в рекуррентных слоях,  число нейронов в полносвязных слоях, параметр в слое Dropuot(), алгоритм обучения.
6. На неразмеченной части набора данных выполнить предобучение с помощью автокодировщика, построенного на основании базового нейросетевого классификатора, заданного в варинте. Выполнить точную настройку (дообучение) базовой нейронной сети на размеченном наборе данных.
7. Сделать выводы по проведенному анализу.

**Вариант 28**

Гибридная нейросетевая модель: последовательностное соединение  одномерных сверточных и рекуррентных двунаправленных (Bidirectional layer)  нейросетевых слоев с полносвязным классификатором: самостоятельная реализация.

## Исходные данные

В качестве исходных данных испольщуется датасет на основе KDD [1], [2]. 

Используемые файлы:

 - kdd_10000_labeled.csv - выборка из 10К записей с меткой атаки (https://github.com/Demetrey/DMaP/blob/CP/CP/datasets/kdd_10000_labeled.csv);
 - kdd_50000_unlabeled.csv - выборка из 50К записей без метки атаки (https://github.com/Demetrey/DMaP/blob/CP/CP/datasets/kdd_50000_unlabeled.csv);
 - Field Names.csv - наименования полей (https://github.com/defcom17/NSL_KDD/blob/master/Field%20Names.csv).

Наюор данных NSL-KDD содержит 4 категории угроз [3]:

 - Denial of Service (dos). Набор атак, в которых злоумышленник ограничивает доступ верифицированным пользователям к конкретному сервису через определенный протокол (Back, Land, Neptune, Pod, Smurf,Teardrop,Apache2, Udpstorm, Processtable, Worm);
 - Remote to Local (r2l). Набор атак, в которых злоумышленник пытается получить доступ извне к локальной машине пользователя (Guess_Password, Ftp_write, Imap, Phf, Multihop, Warezmaster, Warezclient, Spy, Xlock, Xsnoop, Snmpguess, Snmpgetattack, Httptunnel, Sendmail, Named);
 - User to Root (u2r). Набор атак, в которых злоумышленник, имея доступ к машине жертвы, пытается получить права более привилегированного пользователя (Buffer_overflow, Loadmodule, Rootkit, Perl, Sqlattack, Xterm, Ps);
 - Probe. Набор атак, в которых злоумышленник пытается получить сведения об инфраструктуре пользователя (Satan, Ipsweep, Nmap, Portsweep, Mscan, Saint).

Заданные выборки предназначены для двоичной классификации: наличие угрозы / отсутствие угрозы (normal).

Размерность вектра составляет 42 (41 атрибут, нименования которых хранится в файле Field Names.csv, 42-ой атрибут представляет собой тип атаки).
43 атрибут NSL-KDD был исключен.

### Загрузка исходных данных

In [2]:
import urllib.request
import os

save_path = './datasets'
download_path = './datasets/download'
urls = {'kdd_10000_labeled': 'https://raw.githubusercontent.com/Demetrey/DMaP/CP/CP/datasets/kdd_10000_labeled.csv',
        'kdd_50000_unlabeled': 'https://raw.githubusercontent.com/Demetrey/DMaP/CP/CP/datasets/kdd_50000_unlabeled.csv'}

if not os.path.exists(save_path):
    os.mkdir(save_path)

for key in urls:
    file_name = os.path.join(save_path, key + '.csv')
    urllib.request.urlretrieve(urls[key], file_name)

# download fields
if not os.path.exists(download_path):
    os.mkdir(download_path)

urllib.request.urlretrieve('https://raw.githubusercontent.com/defcom17/NSL_KDD/master/Field%20Names.csv',
                           os.path.join(download_path, 'fields.csv'))


('./datasets/download\\fields.csv', <http.client.HTTPMessage at 0x2d8c29e4af0>)

### Просмотр исходных данных

#### Объединение заголовков и данных

Просмотр тестовых данных совместно со сформированными заголовками таблицы

In [20]:
import pandas as pd
data = pd.read_csv('./datasets/kdd_10000_labeled.csv')
data.columns = pd.concat([pd.read_csv('./datasets/download/fields.csv', header=None), pd.DataFrame({0: ['Attack type']})]).T.iloc[0]

In [23]:
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack type
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


#### Пояснение признаков

## Список использованных источников

1. Подготовленные данные (https://github.com/Demetrey/DMaP/tree/CP/CP/datasets).
2. Датасет NSL-KDD (https://github.com/defcom17/NSL_KDD).
2. Е.М. Бурлаков Применение метода анализа соответствий для оптимизации комбинаций атрибутов у наборов данных. Вестник ПНИПУ. Электротехника, информационные технологии, системы управления №26, 2018.